In [4]:
%run partialOrder.ipynb

R = {(1, 4),(1, 5),(2, 4),(3, 6)}
S = {(4, 9),(5, 8),(6, 7)}
R∘S = {(1, 8),(1, 9),(2, 9),(3, 7)}

is binary relation? R = True
is binary relation? S = True

is homogeneous relation? R = False
is homogeneous relation? S = False

is identity relation? R = False
is identity relation? S = False

S[A] = FiniteSet(8, 9)
S-1[A] = FiniteSet(5, 6)

dom S = FiniteSet(4, 5, 6)
rng S = FiniteSet(7, 8, 9)

R == S: False
R != S: True
R < S: False
R <= S: False
R > S: False
R >= S: False

R | S = FiniteSet((1, 4), (1, 5), (2, 4), (3, 6), (4, 9), (5, 8), (6, 7))
R & S = EmptySet
R - S = FiniteSet((1, 4), (1, 5), (2, 4), (3, 6))
R ^ S = FiniteSet((1, 4), (1, 5), (2, 4), (3, 6), (4, 9), (5, 8), (6, 7))
R * S = ProductSet(FiniteSet((1, 4), (1, 5), (2, 4), (3, 6)), FiniteSet((4, 9), (5, 8), (6, 7)))

R complement = {(1, 6),(2, 5),(2, 6),(3, 4),(3, 5)}
R∘S converse = {(7, 3),(8, 1),(9, 1),(9, 2)}

'1' is in relation with: FiniteSet(4, 5) in R
FiniteSet(4, 5)
0
False
False
False
False
True
R = {(1, 1),(2, 1

In [5]:
class Syntax:
    
    def execute(self):
        ...
        
class Constant(Syntax):
    
    def __init__(self, value):
        self.value = value
        
    def __repr__(self):
        return str(self.value)
    
    def execute(self):
        return self.value
        
class BinaryOperations(Syntax):
    
    def __init__(self, left, right):
        self.left = left
        self.right = right
        
class Union(BinaryOperations):
    
    def execute(self):
        return self.left.execute() | self.right.execute()
    
class Intersection(BinaryOperations):
    
    def execute(self):
        return self.left.execute() & self.right.execute()
    
class Difference(BinaryOperations):
    
    def execute(self):
        return self.left.execute() - self.right.execute()
    
class SymmetricDifference(BinaryOperations):
    
    def execute(self):
        return self.left.execute() ^ self.right.execute()
        
class Equal(BinaryOperations):
    
    def execute(self):
        return self.left.execute() == self.right.execute()
    
class NotEqual(BinaryOperations):
    
    def execute(self):
        return self.left.execute() != self.right.execute()
    
class LowerThan(BinaryOperations):
    
    def execute(self):
        return self.left.execute() < self.right.execute()
    
class LowerEqual(BinaryOperations):
    
    def execute(self):
        return self.left.execute() <= self.right.execute()
    
class GreaterThan(BinaryOperations):
    
    def execute(self):
        return self.left.execute() > self.right.execute()
    
class GreaterEqual(BinaryOperations):
    
    def execute(self):
        return self.left.execute() >= self.right.execute()
    
class Multiplication(BinaryOperations):
    
    def execute(self):
        tmp = self.left.execute() * self.right.execute()
        result = set()
        for x in tmp:
            result.add(x)
        return FiniteSet(*result)
    
class Conjunction(BinaryOperations):
    
    def execute(self):
        return self.left.execute() and self.right.execute()
    
class Disjunction(BinaryOperations):
    
    def execute(self):
        return self.left.execute() or self.right.execute()
    
class Implication(BinaryOperations):
    
    def execute(self):
        return (not (self.left.execute())) or self.right.execute()        

In [6]:
class Interpreter():
    
    def __init__(self, text=""):
        self._index = 0
        self._input = text
        self._look = ''
        self._token = ""
        self._position = 0
        
    def reset(self):
        self._index = 0
        self._position = 0
        self.__next()
        self.scan()
    
    def __next(self):
        if self._index >= len(self._input):
            self._look = '\0'
        else:
            self._look = self._input[self._index]
            self._index = self._index + 1

    def scan(self):

        while self._look == ' ' or self._look == '\n':
            self.__next()
            
        self._token = ""
        self._position = self._index - 1
        
        if self._look.isalpha():
            while self._look.isalpha():
                self._token = self._token + self._look
                self.__next()
        elif self._look in ['<','>','!','-','=']:
            self._token = self._look
            self.__next()
            if self._look in ['=','>']:
                self._token += self._look
                self.__next()
        elif self._look not in ['\0']:
            self._token = self._look
            self.__next()
        else:
            self._token = "\0"
            
    def show(self):
        self.reset()
        while True:
            self.scan()
            if self._token != "\0":
                print(f"- {self._token}")
            else:
                break
       
    def check(self, expected_token):
        if expected_token != self._token:
            raise Exception(f"Unexpected token - {self._token} detected! Expected token - {expected_token}")
    
    def __constant(self):
        try:
            result = Constant(globals()[self._token])
            self.scan()
            return result
        except:
            raise Exception(f"Global variable - '{self._token}' doesn't exist!")
            
    def __braces(self):
        if self._token != "(":
            return self.__constant()
        self.scan()
        result = self.__operator()
        self.check(")")
        self.scan()
        return result
        
    def __operator(self):
        result = self.__braces()
        while True:
            if self._token == "&":
                self.scan()
                result = Intersection(result, self.__braces())
            elif self._token == "|":
                self.scan()
                result = Union(result, self.__braces())
            elif self._token == "-":
                self.scan()
                result = Difference(result, self.__braces())
            elif self._token == "^":
                self.scan()
                result = SymmetricDifference(result, self.__braces())
            elif self._token == "==":
                self.scan()
                result = Equal(result, self.__braces())
            elif self._token == "!=":
                self.scan()
                result = NotEqual(result, self.__braces())
            elif self._token == ">":
                self.scan()
                result = GreaterThan(result, self.__braces())
            elif self._token == "<":
                self.scan()
                result = LowerThan(result, self.__braces())
            elif self._token == ">=":
                self.scan()
                result = GreaterEqual(result, self.__braces())
            elif self._token == "<=":
                self.scan()
                result = LowerEqual(result, self.__braces())
            elif self._token == "*":
                self.scan()
                result = Multiplication(result, self.__braces())
            elif self._token == "and":
                self.scan()
                result = Conjunction(result, self.__braces())
            elif self._token == "or":
                self.scan()
                result = Disjunction(result, self.__braces())
            elif self._token == "->":
                self.scan()
                result = Implication(result, self.__braces())
            else:
                return result
            
    def parse(self):
        self.reset()
        return self.__operator()

In [7]:
A = FiniteSet(1,2,3,4)
B = FiniteSet(3,4,5,6)

tree1 = Multiplication(Intersection(Union(Constant(A), Constant(B)),Difference(Constant(A), Constant(B))), Constant(B))
tree2 = Multiplication(SymmetricDifference(Constant(A), Constant(B)), Constant(B))
tree3 = LowerEqual(tree1, tree2)

print(tree1.execute())
print(tree2.execute())
print(tree3.execute())

FiniteSet((1, 3), (1, 4), (1, 5), (1, 6), (2, 3), (2, 4), (2, 5), (2, 6))
FiniteSet((1, 3), (1, 4), (1, 5), (1, 6), (2, 3), (2, 4), (2, 5), (2, 6), (5, 3), (5, 4), (5, 5), (5, 6), (6, 3), (6, 4), (6, 5), (6, 6))
True


In [8]:
i = Interpreter("(((A & B) <= (A | B)) or ((A*B) > (A*B))) -> (((A != A) and (B == B)) -> (A >= (B - (A ^ B))))")
i.show()
tree = i.parse()

- (
- (
- A
- &
- B
- )
- <=
- (
- A
- |
- B
- )
- )
- or
- (
- (
- A
- *
- B
- )
- >
- (
- A
- *
- B
- )
- )
- )
- ->
- (
- (
- (
- A
- !=
- A
- )
- and
- (
- B
- ==
- B
- )
- )
- ->
- (
- A
- >=
- (
- B
- -
- (
- A
- ^
- B
- )
- )
- )
- )


In [9]:
print(tree.execute())

True
